Романовский А.Н. 02.11.2022

Использовать полученные знания в части обучения моделей для обучения и подбора параметров в задачах из предыдущих ДЗ на выбор (мфо, задача с тем возьмёт ли человек кредит или нет)
В частности использовать:


GaussianNB
LinearRegression + PolynomialFeatures.
При обучении использовать make_pipeline
GridSearchCV для поиска параметров

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB


import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('MFOcredit.csv', sep = ';', parse_dates=True)

In [3]:
df.head(1)

,id,date_start,date_end,gender,age,auto,housing,marstatus,regclient,jobtype,region,credits,children,delinq60plus
0,1,03-Jan-2013,12-Jan-2013,Мужской,44,Нет,Собственное,Гражданский брак/женат/замужем,Нет,Официальное,Новосибирская область,Нет,Да,Нет


In [4]:
df = df.drop(columns = ['id'],axis = 1)
df.delinq60plus = np.where(df.delinq60plus=='Нет', 0, 1)

In [5]:
df['date_start'] = pd.to_datetime(df['date_start'])
df['date_end'] = pd.to_datetime(df['date_end'])

In [6]:
viewDate = pd.to_datetime(df['date_start']) 
df['year'] = viewDate.dt.year.astype("category") 
df['quarter'] = viewDate.dt.quarter.astype("category")
df['month'] = viewDate.dt.month.astype("category")
df['weekday'] = viewDate.dt.dayofweek.astype("category")

In [7]:
df['credit_period']=(df['date_end']-df['date_start']).dt.days

In [39]:
df.head(2)

,gender,age,auto,regclient,credits,children,year,quarter,month,weekday,...,marstatus_Гражданский брак/женат/замужем,marstatus_Разведен,marstatus_Холост,jobtype_Неофициальное,jobtype_Официальное,region_Алтайский край,region_Кемеровская область север,region_Кемеровская область юг,region_Красноярский край,region_Новосибирская область
0,1,44,0,0,0,1,2013,1,1,3,...,1,0,0,0,1,0,0,0,0,1
1,1,21,0,0,1,0,2013,1,1,3,...,0,0,1,0,1,0,0,1,0,0


In [9]:
df = df.drop(['date_end', 'date_start' ],   axis = 1)

In [10]:
df.loc[(df['age'] < 21) & (df['gender'] == 'Мужской') & (df['housing'] == 'Пропуск поля'), 'housing'] = 'Живут с родителями'
df.loc[(df['age'] < 25) & (df['gender'] == 'Мужской') & (df['housing'] == 'Пропуск поля'), 'housing'] = 'Арендуемое'
df.loc[(df['age'] < 51) & (df['gender'] == 'Мужской') & (df['housing'] == 'Пропуск поля'), 'housing'] = 'Собственное'
df.loc[(df['age'] < 74) & (df['gender'] == 'Мужской') & (df['housing'] == 'Пропуск поля'), 'housing'] = 'Долевая собственность'

df.loc[(df['age'] < 20) & (df['gender'] == 'Женский') & (df['housing'] == 'Пропуск поля'), 'housing'] = 'Живут с родителями'
df.loc[(df['age'] < 27) & (df['gender'] == 'Женский') & (df['housing'] == 'Пропуск поля'), 'housing'] = 'Арендуемое'
df.loc[(df['age'] < 30) & (df['gender'] == 'Женский') & (df['housing'] == 'Пропуск поля'), 'housing'] = 'Долевая собственность'
df.loc[(df['age'] < 75) & (df['gender'] == 'Женский') & (df['housing'] == 'Пропуск поля'), 'housing'] = 'Собственное'

In [11]:
df.loc[(df['age'] < 36) & (df['gender'] == 'Мужской') & (df['marstatus'] == 'Пропуск поля'), 'marstatus'] = 'Холост'
df.loc[(df['age'] < 50) & (df['gender'] == 'Мужской') & (df['marstatus'] == 'Пропуск поля'), 'marstatus'] = 'Гражданский брак/женат/замужем'
df.loc[(df['age'] < 75) & (df['gender'] == 'Мужской') & (df['marstatus'] == 'Пропуск поля'), 'marstatus'] = 'Вдова/вдовец'

df.loc[(df['age'] < 39) & (df['gender'] == 'Женский') & (df['marstatus'] == 'Пропуск поля'), 'marstatus'] = 'Холост'
df.loc[(df['age'] < 51) & (df['gender'] == 'Женский') & (df['marstatus'] == 'Пропуск поля'), 'marstatus'] = 'Гражданский брак/женат/замужем' 
df.loc[(df['age'] < 75) & (df['gender'] == 'Женский') & (df['marstatus'] == 'Пропуск поля'), 'marstatus'] = 'Вдова/вдовец' 

In [12]:
df.loc[(df['gender'] == 'Женский') & (df['credits'] == 'Пропуск поля'), 'credits'] = 'Да'
df.loc[(df['gender'] == 'Мужской') & (df['credits'] == 'Пропуск поля'), 'credits'] = 'Нет'

In [13]:
df.loc[(df['age'] < 45) & (df['gender'] == 'Мужской') & (df['children'] == 'Пропуск поля'), 'children'] = 'Нет'
df.loc[(df['age'] < 75) & (df['gender'] == 'Мужской') & (df['children'] == 'Пропуск поля'), 'children'] = 'Да'

df.loc[(df['age'] < 31) & (df['gender'] == 'Женский') & (df['children'] == 'Пропуск поля'), 'children'] = 'Нет'
df.loc[(df['age'] < 75) & (df['gender'] == 'Женский') & (df['children'] == 'Пропуск поля'), 'children'] = 'Да'

In [14]:
df.head(2)

,gender,age,auto,housing,marstatus,regclient,jobtype,region,credits,children,delinq60plus,year,quarter,month,weekday,credit_period
0,Мужской,44,Нет,Собственное,Гражданский брак/женат/замужем,Нет,Официальное,Новосибирская область,Нет,Да,0,2013,1,1,3,9
1,Мужской,21,Пропуск поля,Живут с родителями,Холост,Нет,Официальное,Кемеровская область юг,Да,Нет,0,2013,1,1,3,14


In [15]:
df.loc[(df.auto == 'Пропуск поля'), 'auto'] = 'Нет'
df.loc[(df.jobtype == 'Пропуск поля'), 'jobtype'] = 'Официальное'
#df.loc[(df.marstatus == 'Пропуск поля'), 'marstatus'] = 'Гражданский брак/женат/замужем'
#df.loc[(df.credits == 'Пропуск поля'), 'credits'] = 'Нет'
#df.loc[(df.children == 'Пропуск поля'), 'children'] = 'Нет'

In [16]:
df.gender = np.where(df.gender=='Женский', 0, 1)
df.regclient = np.where(df.regclient=='Нет', 0, 1)
df.credits = np.where(df.credits=='Нет', 0, 1)
df.children = np.where(df.children=='Нет', 0, 1)
df.auto = np.where(df.auto=='Нет', 0, 1)

In [17]:
df = pd.concat([df, 
                      pd.get_dummies(df['housing'], prefix="housing"),
                      pd.get_dummies(df['marstatus'], prefix="marstatus"),
                      pd.get_dummies(df['jobtype'], prefix="jobtype"),
                      pd.get_dummies(df['region'], prefix="region")],
                      axis = 1)

In [18]:
df = df.drop(['housing','marstatus','jobtype','region'],   axis = 1)

In [19]:
df.head(1)

,gender,age,auto,regclient,credits,children,delinq60plus,year,quarter,month,...,marstatus_Гражданский брак/женат/замужем,marstatus_Разведен,marstatus_Холост,jobtype_Неофициальное,jobtype_Официальное,region_Алтайский край,region_Кемеровская область север,region_Кемеровская область юг,region_Красноярский край,region_Новосибирская область
0,1,44,0,0,0,1,0,2013,1,1,...,1,0,0,0,1,0,0,0,0,1


In [20]:
y = df['delinq60plus']
df = df.drop(['delinq60plus'],   axis = 1)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(
      df, y, test_size=0.3, random_state=42)

### GaussianNB

In [22]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)

GaussianNB()

In [23]:
y_pred = gnb.predict(X_test)
roc_auc_score(y_test, y_pred)

0.6399745020660446

In [24]:
GaussianNB().get_params().keys()

dict_keys(['priors', 'var_smoothing'])

In [25]:
parametrs = {'var_smoothing': np.logspace(0,-9, num=100)}

grid = GridSearchCV(gnb, parametrs).fit(X_train, y_train)

grid.best_params_

{'var_smoothing': 0.0001873817422860383}

In [26]:
gnb = grid.best_estimator_
gnb.fit(X_train, y_train)

GaussianNB(var_smoothing=0.0001873817422860383)

In [27]:
y_pred = gnb.predict(X_test)
roc_auc_score(y_test, y_pred)

0.7127530783197327

### LinearRegression

In [28]:
LinearRegression().get_params().keys()

dict_keys(['copy_X', 'fit_intercept', 'n_jobs', 'normalize', 'positive'])

In [29]:
model = LinearRegression(fit_intercept=True)
model.fit(X_train, y_train)

LinearRegression()

In [30]:
y_pred = model.predict(X_test)
roc_auc_score(y_test, y_pred)

0.8360666715013003

### PolynomialFeatures


In [31]:
poly_model = make_pipeline(PolynomialFeatures(),
                           LinearRegression())

In [32]:
poly_model.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'polynomialfeatures', 'linearregression', 'polynomialfeatures__degree', 'polynomialfeatures__include_bias', 'polynomialfeatures__interaction_only', 'polynomialfeatures__order', 'linearregression__copy_X', 'linearregression__fit_intercept', 'linearregression__n_jobs', 'linearregression__normalize', 'linearregression__positive'])

In [33]:
parametrs = {'polynomialfeatures__degree': range(2, 4)}

grid = GridSearchCV(poly_model, parametrs).fit(X_train, y_train)

grid.best_params_

{'polynomialfeatures__degree': 2}

In [34]:
poly_model = grid.best_estimator_
poly_model.fit(X_train, y_train)

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures()),
                ('linearregression', LinearRegression())])

In [35]:
y_pred = poly_model.predict(X_test)
roc_auc_score(y_test, y_pred)

0.8517471204903004

### RandomForestClassifier

In [36]:
rfc = RandomForestClassifier(max_depth = 10, 
                             n_estimators = 300, 
                             max_features = 10,
                             min_samples_split= 10,
                             min_samples_leaf = 10)
rfc.fit(X_train, y_train)


RandomForestClassifier(max_depth=10, max_features=10, min_samples_leaf=10,
                       min_samples_split=10, n_estimators=300)

In [37]:
y_pred = rfc.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, y_pred)

0.856566526938141

In [38]:
feature_names = X_train.columns
importances = rfc.feature_importances_
indices = np.argsort(importances)[::-1]

print("Важность предикторов:")
for f, idx in enumerate(indices):
    print("{:2d}. '{:5s}' ({:.4f})".format(f + 1,
                                           feature_names[idx],
                                           importances[idx]))

Важность предикторов:
 1. 'month' (0.4229)
 2. 'quarter' (0.2592)
 3. 'age  ' (0.1180)
 4. 'credits' (0.0331)
 5. 'credit_period' (0.0327)
 6. 'regclient' (0.0213)
 7. 'region_Кемеровская область север' (0.0188)
 8. 'weekday' (0.0161)
 9. 'year ' (0.0156)
10. 'region_Новосибирская область' (0.0075)
11. 'gender' (0.0061)
12. 'marstatus_Холост' (0.0058)
13. 'region_Кемеровская область юг' (0.0057)
14. 'region_Алтайский край' (0.0052)
15. 'children' (0.0048)
16. 'housing_Собственное' (0.0045)
17. 'marstatus_Гражданский брак/женат/замужем' (0.0044)
18. 'region_Красноярский край' (0.0028)
19. 'housing_Живут с родителями' (0.0024)
20. 'marstatus_Вдова/вдовец' (0.0024)
21. 'auto ' (0.0023)
22. 'marstatus_Разведен' (0.0022)
23. 'housing_Долевая собственность' (0.0020)
24. 'housing_Арендуемое' (0.0018)
25. 'housing_Муниципальное' (0.0010)
26. 'jobtype_Официальное' (0.0008)
27. 'jobtype_Неофициальное' (0.0008)


#### Вывод:
Лучшие результаты с подбором параметров: 
* RandomForestClassifier - 0.8565
* PolynomialFeatures - 0.8517
* LinearRegression - 0.8360
* GaussianNB - 0.7127

Замена пропусков с привязкой к возрасту дала очень маленький процент к точности моделей, улучшился 3-й знак после запятой.

